In [1]:
# =============================== #
# By-Price LGBM: SUBMIT (Stage-B) #
# - price models -> log-returns   #
# - strict output format          #
# - seed store with exact lag dates
# =============================== #

import os, json, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import polars as pl
import lightgbm as lgb
import kaggle_evaluation.mitsui_inference_server

# ---------------- Paths ----------------
DATA_PATH = "/kaggle/input/mitsui-commodity-prediction-challenge"
MODEL_INPUT_DIR = "/kaggle/input/mitsui-byprice-lgbm-v1/models_price"  # ←あなたのDataset名に合わせる

# ---------------- Utils ----------------
def preprocess_for_lgbm(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    obj = df.select_dtypes(include=["object"]).columns
    if len(obj) > 0:
        df[obj] = df[obj].apply(pd.to_numeric, errors="coerce")
    for c in df.select_dtypes(include=["category"]).columns:
        df[c] = df[c].cat.codes
    return df

def parse_pair_string(s: str):
    s = str(s)
    if " - " in s:
        a, b = s.split(" - ", 1)
        return a.strip(), b.strip()
    return s.strip(), None

# ---------------- Load meta & models ----------------
with open(os.path.join(MODEL_INPUT_DIR, "meta_price.json"), "r") as f:
    meta = json.load(f)

feat_cols_base = meta["feat_cols_base"]
price_cols     = meta["price_cols"]
use_log_price  = bool(meta.get("use_log_price", True))
max_lag        = int(meta.get("max_lag", 7))

price_models: dict[str, lgb.Booster] = {}
for pcol in price_cols:
    path = os.path.join(MODEL_INPUT_DIR, f"price__{pcol}.txt")
    if os.path.exists(path):
        price_models[pcol] = lgb.Booster(model_file=path)

# ---------------- Labels & target map ----------------
labels = pd.read_csv(f"{DATA_PATH}/train_labels.csv")
label_cols = [c for c in labels.columns if c != "date_id"]
assert len(label_cols) == (labels.shape[1] - 1)

pairs = pd.read_csv(f"{DATA_PATH}/target_pairs.csv")
target_map = {}
for _, row in pairs.iterrows():
    a, b = parse_pair_string(row["pair"])
    target_map[str(row["target"])] = (a, b, int(row["lag"]))

# ---------------- Rolling price store ----------------
from collections import deque, defaultdict
class RollingPriceStore:
    def __init__(self, max_lag: int):
        self.max_lag = max_lag
        self.buf = defaultdict(deque)  # price_col -> deque[(date_id, price)]
    def push(self, price_col: str, date_id: int, price: float):
        dq = self.buf[price_col]
        if dq and dq[-1][0] == date_id:
            dq[-1] = (date_id, price)
        else:
            dq.append((date_id, price))
            while len(dq) > self.max_lag + 64:
                dq.popleft()
    def lookup(self, price_col: str, date_id: int):
        dq = self.buf.get(price_col, None)
        if not dq: return None
        # exact match のみ（ここは exact でOK。代わりに事前に exact seed を入れる）
        for d,p in reversed(dq):
            if d == date_id:
                return p
        return None
    def lookup_lag(self, price_col: str, curr_date: int, L: int):
        return self.lookup(price_col, curr_date - L)

store = RollingPriceStore(max_lag=max_lag)

# ---------- NEW: exact lag seeding using train tail ----------
# 1) train を読み込み、price_cols の最後の実測値を取得
train_path = os.path.join(DATA_PATH, "train.csv")
usecols = ["date_id"] + [c for c in price_cols if c not in ("date_id",)]
train_all = pd.read_csv(train_path, usecols=usecols)
train_all = preprocess_for_lgbm(train_all)
train_all = train_all.sort_values("date_id").reset_index(drop=True)
last_train_date = int(train_all["date_id"].iloc[-1])

# 2) test の最初の date_id（= 最初に predict が呼ばれる想定日）を取得
test_head = pd.read_csv(os.path.join(DATA_PATH, "test.csv"), usecols=["date_id"]).sort_values("date_id")
first_test_date = int(test_head["date_id"].iloc[0])

# 3) 各 pcol について「最後の正の実測値」を取得
last_val_per_col = {}
for pcol in price_cols:
    if pcol in train_all.columns:
        v = pd.to_numeric(train_all[pcol], errors="coerce").dropna()
        v = v[v > 0]
        if len(v) > 0:
            last_val_per_col[pcol] = float(v.iloc[-1])

# 4) exact な lag 日付に“同じ終値”を埋める（carry-forward seed）
#    これで lookup_lag(pcol, first_test_date, k) が必ずヒットする
for pcol, v in last_val_per_col.items():
    for k in range(1, max_lag + 1):
        store.push(pcol, first_test_date - k, v)

# （オプション）train の末尾実測をそのまま入れておく（将来使うため）
tail_dates = train_all["date_id"].unique()[-(max_lag+8):]
for d in tail_dates:
    row = train_all[train_all["date_id"] == d]
    for pcol in price_cols:
        if pcol in row.columns:
            vv = pd.to_numeric(row[pcol], errors="coerce").dropna()
            if len(vv) > 0 and np.isfinite(vv.iloc[0]) and vv.iloc[0] > 0:
                store.push(pcol, int(d), float(vv.iloc[0]))

# ---------------- Output guard ----------------
def _strict_output(out_dict: dict, label_cols: list[str]) -> pd.DataFrame:
    vals = []
    for c in label_cols:
        v = out_dict.get(c, 0.0)
        if v is None or not np.isfinite(v):
            v = 0.0
        vals.append(float(v))
    arr = np.asarray(vals, dtype="float64")
    arr = np.clip(arr, -50.0, 50.0)
    df = pd.DataFrame([arr], columns=label_cols)
    df.index.name = None
    df = df.replace([np.inf, -np.inf], 0.0).fillna(0.0)
    return df

# ---------------- Core predict() ----------------
def predict(test_batch: pl.DataFrame | pd.DataFrame, lag1, lag2, lag3, lag4) -> pd.DataFrame:
    Xb_raw = test_batch.to_pandas() if isinstance(test_batch, pl.DataFrame) else test_batch
    assert "date_id" in Xb_raw.columns, "date_id が見つかりません"
    date_t = int(pd.to_numeric(Xb_raw["date_id"], errors="coerce").max())

    Xb_raw = preprocess_for_lgbm(Xb_raw)

    # 1) 当日価格の取得（実測優先 → 無ければモデル予測）
    P_t = {}
    for pcol in price_cols:
        val = None
        if pcol in Xb_raw.columns:
            v = pd.to_numeric(Xb_raw[pcol], errors="coerce").dropna()
            if len(v) > 0:
                val = float(v.mean())
        if val is None:
            booster = price_models.get(pcol)
            if booster is not None:
                feat_cols = [c for c in feat_cols_base if c != pcol and c in Xb_raw.columns]
                if len(feat_cols) > 0:
                    yhat = booster.predict(Xb_raw[feat_cols])
                    m = float(np.asarray(yhat, dtype="float64").mean())
                    val = float(np.exp(m)) if use_log_price else m
        if val is not None and np.isfinite(val) and val > 0:
            P_t[pcol] = val
            store.push(pcol, date_t, val)

    # 2) targetごとに log-return 合成（exact lag が必ずヒットする）
    out = {}
    for tgt in label_cols:
        a, b, L = target_map.get(tgt, (None, None, None))
        if a is None or L is None:
            out[tgt] = 0.0
            continue
        P_today = store.lookup(a, date_t) or P_t.get(a, None)
        base_col = b if b else a
        P_lag = store.lookup_lag(base_col, date_t, L)  # ← ここが必ずヒットするよう事前に exact seed 済み

        if P_today is None or P_lag is None or P_today <= 0 or P_lag <= 0:
            out[tgt] = 0.0
        else:
            out[tgt] = float(np.log(P_today) - np.log(P_lag))

    return _strict_output(out, label_cols)

# ---------------- Serve ----------------
server = kaggle_evaluation.mitsui_inference_server.MitsuiInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    server.serve()
else:
    server.run_local_gateway((DATA_PATH,))


In [2]:
# pq_path  = "/kaggle/working/submission.parquet"
# csv_out  = "/kaggle/working/submission_from_parquet.csv"

# # 1) target_map が全ターゲットを網羅しているか
# missing_tgts = [t for t in label_cols if t not in target_map]
# print("target_mapに無いtarget数:", len(missing_tgts))  # 0 が理想

# # 2) ラベル本数と返却本数の突合テスト
# _df = _strict_output({}, label_cols)
# print("返却列数:", _df.shape[1], "(期待:", len(label_cols), ")  OK?", _df.shape[0]==1)

# # ===== Inspect submission artifact (Parquet -> CSV for inspection) =====
# # Run if necessary to check
# sub_pl = pl.read_parquet(pq_path)
# print(f"[OK] Loaded Parquet: {pq_path} shape={sub_pl.shape}")

# # Export CSV for confirmation (not for submission)
# sub_pl.write_csv(csv_out)
# print(f"[OK] Wrote CSV for inspection: {csv_out}")

# sub = sub_pl.to_pandas()
# pd.set_option("display.max_columns", 30)
# sub.head()